# Setting up environment

In [1]:
!pip install timm==0.4.12
!pip install fvcore einops submitit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: timm
    Found existing installation: timm 1.0.11
    Uninstalling timm-1.0.11:
      Successfully uninstalled timm-1.0.11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 4.6 MB/s eta 0:00:00
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=a12a039f1078db324b71a4ee08979f1ff9f3898625ee2e6b3d9cd3dfecc60463
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=36ae42edc189dc2eb2

# Imports and Constants

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, roc_auc_score, average_precision_score, confusion_matrix
import pandas as pd
import gc

# Paths to the dataset
data_dir = '/kaggle/input/brain-tumor-classification-mri'
train_dir = os.path.join(data_dir, 'Training')
test_dir = os.path.join(data_dir, 'Testing')

# Parameters
num_classes = 4
batch_size = 16
num_epochs = 20
learning_rate = 1e-4
val_split = 0.2  # 20% of combined training data for validation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset Preprocessing

In [3]:
# Normalization
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Data Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Add color jitter
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),  # Random affine transformations
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Random zoom
    transforms.ToTensor(),
    normalize
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

# Dataset Loading

In [4]:
# Load the datasets
full_train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)

# Split the dataset into training and validation
train_size = int((1 - val_split) * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Override transform for validation dataset
val_dataset.dataset.transform = val_test_transform

# Test dataset
test_dataset = datasets.ImageFolder(root=test_dir, transform=val_test_transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Cloning Cross-ViT git and Importing models

In [5]:
!git clone https://github.com/IBM/CrossViT.git
import sys
sys.path.append("CrossViT")

from models.crossvit import crossvit_9_224, crossvit_15_224, crossvit_18_224, crossvit_9_dagger_224, crossvit_15_dagger_224, crossvit_15_dagger_384, crossvit_18_dagger_224, crossvit_18_dagger_384

# Models to evaluate
model_names = [
    "crossvit_9_224",
    "crossvit_15_224",
    "crossvit_18_224",
    "crossvit_9_dagger_224",
    "crossvit_15_dagger_224",
    "crossvit_15_dagger_384",
    "crossvit_18_dagger_224",
    "crossvit_18_dagger_384"
]

models = [
    crossvit_9_224,
    crossvit_15_224,
    crossvit_18_224,
    crossvit_9_dagger_224,
    crossvit_15_dagger_224,
    crossvit_15_dagger_384,
    crossvit_18_dagger_224,
    crossvit_18_dagger_384
]

Cloning into 'CrossViT'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 64 (delta 24), reused 43 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 33.12 KiB | 8.28 MiB/s, done.
Resolving deltas: 100% (24/24), done.


# Training and Evaluation function

In [6]:
# Metrics to store
metrics = []

# Training and Evaluation Functions
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    train_bar = tqdm(loader, desc="Training", leave=False)
    for inputs, labels in train_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        train_bar.set_postfix(loss=loss.item())

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_preds = []
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_preds = []

    val_bar = tqdm(loader, desc="Validation", leave=False)
    with torch.no_grad():
        for inputs, labels in val_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc, all_labels, all_preds

# Training and Testing Loop

In [ ]:
# Training Loop for Multiple Models
ensemble_predictions = []  # To store all models' predictions for ensemble
ensemble_labels = []  # Ground truth labels for comparison
for model_name, model_fn in zip(model_names, models):
    torch.cuda.empty_cache()
    print(f"Evaluating {model_name}")
    model = model_fn(pretrained=False, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    train_losses, val_losses = [], []
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        scheduler.step()  # Adjust learning rate
        print(f"Epoch [{epoch+1}/{num_epochs}] for {model_name}")
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc, val_labels, val_preds = evaluate(model, val_loader, criterion, device)

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.4f}, Val Acc:   {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"best_{model_name}.pth")
            print("  [*] Saved best model.")

    # Test the Best Model
    model.load_state_dict(torch.load(f"best_{model_name}.pth"))
    test_loss, test_acc, test_labels, test_preds = evaluate(model, test_loader, criterion, device)

    # Append model predictions for ensemble
    ensemble_predictions.append(test_preds)
    if not ensemble_labels:
        ensemble_labels = test_labels  # Use ground truth labels from the first model

    # Calculate Metrics
    precision = precision_score(test_labels, test_preds, average='weighted')
    recall = recall_score(test_labels, test_preds, average='weighted')
    auroc = roc_auc_score(test_labels, nn.functional.one_hot(torch.tensor(test_preds), num_classes=num_classes), multi_class='ovr')
    #aupr = average_precision_score(test_labels, nn.functional.one_hot(torch.tensor(test_preds), num_classes=num_classes), average='weighted')

    # Confusion Matrix
    cm = confusion_matrix(test_labels, test_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(num_classes), yticklabels=range(num_classes))
    plt.title(f"Confusion Matrix for {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.savefig(f"confusion_matrix_{model_name}.png")
    plt.close()

    # Store Metrics
    metrics.append({
        "Model": model_name,
        "Test Accuracy": test_acc,
        "Precision": precision,
        "Recall": recall,
        "AUROC": auroc,
    })

    # Plot Loss Curves
    plt.figure()
    plt.plot(range(num_epochs), train_losses, label="Training Loss")
    plt.plot(range(num_epochs), val_losses, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(f"Loss Curves for {model_name}")
    plt.legend()
    plt.savefig(f"loss_curves_{model_name}.png")
    plt.close()
    gc.collect()
    torch.cuda.empty_cache()

Evaluating crossvit_9_224


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch [1/20] for crossvit_9_224


  Train Loss: 1.0350, Train Acc: 0.5638
  Val Loss:   0.9698, Val Acc:   0.6038
  [*] Saved best model.
Epoch [2/20] for crossvit_9_224


  Train Loss: 0.8349, Train Acc: 0.6779
  Val Loss:   0.7853, Val Acc:   0.6867
  [*] Saved best model.
Epoch [3/20] for crossvit_9_224


...

  Train Loss: 0.1398, Train Acc: 0.9568
  Val Loss:   0.2741, Val Acc:   0.9121
Epoch [15/20] for crossvit_15_dagger_224


  Train Loss: 0.1362, Train Acc: 0.9596
  Val Loss:   0.2736, Val Acc:   0.9121
Epoch [16/20] for crossvit_15_dagger_224


  Train Loss: 0.1357, Train Acc: 0.9596
  Val Loss:   0.2736, Val Acc:   0.9126
  [*] Saved best model.
Epoch [17/20] for crossvit_15_dagger_224


  Train Loss: 0.1353, Train Acc: 0.9593
  Val Loss:   0.2735, Val Acc:   0.9126
Epoch [18/20] for crossvit_15_dagger_224


Training:   5%|▌         | 26/495 [00:04<01:15,  6.18it/s, loss=0.232] 

# Ensemble Majority Voting

In [ ]:
# Ensemble Majority Voting
ensemble_predictions = np.array(ensemble_predictions)
final_predictions = []
for i in range(len(ensemble_labels)):
    final_predictions.append(np.bincount(ensemble_predictions[:, i]).argmax())

# Calculating Metrics

In [ ]:
# Calculate ensemble metrics
ensemble_accuracy = np.mean(np.array(final_predictions) == np.array(ensemble_labels))
ensemble_precision = precision_score(ensemble_labels, final_predictions, average='weighted')
ensemble_recall = recall_score(ensemble_labels, final_predictions, average='weighted')
ensemble_auroc = roc_auc_score(ensemble_labels, nn.functional.one_hot(torch.tensor(final_predictions), num_classes=num_classes), multi_class='ovr')
#ensemble_aupr = average_precision_score(ensemble_labels, nn.functional.one_hot(torch.tensor(final_predictions), num_classes=num_classes), average='weighted')

# Confusion Matrix for Ensemble
ensemble_cm = confusion_matrix(ensemble_labels, final_predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(ensemble_cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(num_classes), yticklabels=range(num_classes))
plt.title("Confusion Matrix for Ensemble")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig("confusion_matrix_ensemble.png")
plt.close()

# Saving Metrics

In [ ]:
# Store Ensemble Metrics
metrics.append({
    "Model": "Ensemble",
    "Test Accuracy": ensemble_accuracy,
    "Precision": ensemble_precision,
    "Recall": ensemble_recall,
    "AUROC": ensemble_auroc,
})

# Save Metrics to CSV
metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv("model_metrics.csv", index=False)
print("Metrics saved to model_metrics.csv")